# Sub-circuit extraction example

In [ ]:
import obi_one as obi
from pathlib import Path

In [ ]:
input_root = Path("../../data/tiny_circuits")
output_root = Path("../../../../obi-output/extracted_subcircuits")

### Run circuit extraction

Extracting sub-circuits consisting of 50% or 100% excitatory neurons of two small example circuits each.

In [ ]:
circuit_extractions_scan_config = obi.CircuitExtractionScanConfig(
                    initialize=obi.CircuitExtractionScanConfig.Initialize(
                        circuit=[obi.Circuit(name="N_10__top_nodes_dim6", path=str(input_root / "N_10__top_nodes_dim6" / "circuit_config.json")),
                                 obi.Circuit(name="N_10__top_rc_nodes_dim2_rc", path=str(input_root / "N_10__top_rc_nodes_dim2_rc" / "circuit_config.json"))],
                        do_virtual=False,
                        create_external=False,
                    ),
                    neuron_set=obi.PredefinedNeuronSet(node_set="Excitatory", sample_percentage=[50, 100]),
                    info=obi.Info(campaign_name="EXC-Extraction", campaign_description="Extraction of percentages of EXC neurons"),
)

grid_scan = obi.GridScanGenerationTask(form=circuit_extractions_scan_config, output_root=str(output_root))
grid_scan.execute()

In [ ]:
# Display the intermediary data
grid_scan.multiple_value_parameters()
grid_scan.coordinate_parameters()
# grid_scan.coordinate_instances(display=True)

In [ ]:
obi.run_tasks_for_generated_scan(grid_scan)

### Check original circuits vs. extracted sub-circuits

In [ ]:
# Input circuits
for circuit in circuit_extractions_scan_config.initialize.circuit:
    n_exc = len(circuit.sonata_circuit.nodes[circuit.default_population_name].ids("Excitatory"))
    n_inh = len(circuit.sonata_circuit.nodes[circuit.default_population_name].ids("Inhibitory"))
    print(f"Circuit '{circuit}': {n_exc + n_inh} neurons ({n_exc} EXC, {n_inh} INH) and {circuit.sonata_circuit.edges[circuit.default_edge_population_name].size} synapses")

In [ ]:
# Extracted sub-circuits
for _idx, _inst in enumerate(grid_scan.single_configs):
    _inst.initialize_coordinate_output_root(output_root)
    cfg = _inst.coordinate_output_root / "circuit_config.json"
    circuit = obi.Circuit(name="", path=str(cfg))
    n_exc = len(circuit.sonata_circuit.nodes[circuit.default_population_name].ids("Excitatory"))
    n_inh = len(circuit.sonata_circuit.nodes[circuit.default_population_name].ids("Inhibitory"))
    print(f"Extracted sub-circuit '{_inst.coordinate_output_root.relative_to(output_root)}': {n_exc + n_inh} neurons ({n_exc} EXC, {n_inh} INH) and {circuit.sonata_circuit.edges[circuit.default_edge_population_name].size if circuit.default_edge_population_name else 0} synapses")